# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119167e+02     1.405216e+00
 * time: 0.5235610008239746
     1     1.119692e+01     8.794230e-01
 * time: 2.0470900535583496
     2    -1.152405e+01     9.149951e-01
 * time: 2.13710618019104
     3    -3.377764e+01     7.149767e-01
 * time: 2.2963461875915527
     4    -4.738557e+01     5.366183e-01
 * time: 2.3977952003479004
     5    -5.678756e+01     1.941509e-01
 * time: 2.502794027328491
     6    -5.974868e+01     1.046804e-01
 * time: 2.5872039794921875
     7    -6.086163e+01     5.016075e-02
 * time: 2.69685697555542
     8    -6.127248e+01     6.174350e-02
 * time: 2.773332118988037
     9    -6.159130e+01     3.684576e-02
 * time: 2.849560022354126
    10    -6.181787e+01     2.999821e-02
 * time: 2.9290921688079834
    11    -6.199701e+01     2.273859e-02
 * time: 3.0193111896514893
    12    -6.206501e+01     1.976431e-02
 * time: 3.1178810596466064
    13    -6.213218e+01     1.402222e-02
 * time: 3.194888114929199
 

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671073
    AtomicLocal         -18.8557699
    AtomicNonlocal      14.8522656
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485382 
    Xc                  -19.3336824

    total               -62.261666459506